In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [ ]:
train = pd.read_csv("FinalDataset.csv")

In [ ]:
train.info()

In [ ]:
train = train.drop(columns=['Unnamed: 0', 'hashtags', 'mentions', 'NOUN', 'PROPN',
       'PRON', 'ADJ', 'DET', 'ADP', 'VERB', 'ADV', 'CONJ', 'INTJ', 'AUX',
       'SCONJ', 'finalCleanText', 'COUNT_NOUN', 'COUNT_PROPN', 'COUNT_PRON',
       'COUNT_ADJ', 'COUNT_DET', 'COUNT_ADP', 'COUNT_VERB', 'COUNT_ADV',
       'COUNT_CONJ', 'COUNT_INTJ', 'COUNT_AUX', 'COUNT_SCONJ',
       'Positive_Score', 'Negative_Score', 'Neutral_Score', 'Compound Score'])

In [ ]:
train.info()

In [ ]:
X_ = train['cleanTextNoPun']
Xn = []
for x in X_:
    Xn.append(str(x))
train["cleanTextStr"] = Xn
X = train["cleanTextStr"]
Y0 = train['STANCE']
Y1 = train['OPINION']
Y2 = train['POL']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X0_train, X0_test, y0_train, y0_test = train_test_split(X, Y0, test_size=0.2)
X1_train, X1_test, y1_train, y1_test = train_test_split(X, Y1, test_size=0.2)
X2_train, X2_test, y2_train, y2_test = train_test_split(X, Y2, test_size=0.2)

In [ ]:
print(X0_train.shape, y0_train.shape)
print(X0_test.shape, y0_test.shape)

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

In [ ]:
# !ls

In [ ]:
# !unzip glove.twitter.27B.zip

In [ ]:
path = "glove.twitter.27B.200d.txt"
f = open(path, encoding ='utf-8')

In [ ]:
embeddings_index = {}
cnt = 0
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float')
    embeddings_index[word] = coefs
f.close()

In [ ]:
def embedding_output(X):
    maxLen = 75
    emb_dim = 50
    embedding_out = np.zeros((X.shape[0], maxLen, emb_dim))

    for ix in X:
        ic = 0
        ix = ix.split()
    for ij in range(len(ix)):
        try:
            embedding_out[ic][ij] = embeddings_index[X[ic][ij].lower()]
        except:
            embedding_out[ic][ij] = np.zeros((50,))
        ic+=1
    return embedding_out

In [ ]:
embedding_matrix_train0 = embedding_output(X0_train)
embedding_matrix_test0 = embedding_output(X0_test)
embedding_matrix_train1 = embedding_output(X1_train)
embedding_matrix_test1 = embedding_output(X1_test)
embedding_matrix_train2 = embedding_output(X2_train)
embedding_matrix_test2 = embedding_output(X2_test)

In [ ]:
print(embedding_matrix_train0.shape)
print(embedding_matrix_test0.shape)
print(embedding_matrix_train1.shape)
print(embedding_matrix_test1.shape)
print(embedding_matrix_train2.shape)
print(embedding_matrix_test2.shape)

In [ ]:
model0 = Sequential()
model0.add(LSTM(32, input_shape=(75,50), return_sequences=True))
model0.add(Dropout(0.5))
model0.add(LSTM(32, return_sequences=False))
model0.add(Dropout(0.5))
model0.add(Dense(3))
model0.add(Activation('softmax'))
model0.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model0.summary()

In [ ]:
from keras.utils import to_categorical
y0_train = to_categorical(y0_train,num_classes=3)
y0_test = to_categorical(y0_test,num_classes=3)

In [ ]:
hist = model0.fit(embedding_matrix_train0, y0_train, epochs=10, batch_size=32, shuffle=True, validation_split=0.15)

In [ ]:
model0.evaluate(embedding_matrix_test0, y0_test)

In [ ]:
############## With label smoothing ######################

In [ ]:
def smooth_labels(labels, factor=0.1):
	# smooth the labels
	labels *= (1 - factor)
	labels += (factor / labels.shape[1])
	# returned the smoothed labels
	return labels

In [ ]:
y_train_hot_smooth = smooth_labels(y0_train)
y_test_hot_smooth = smooth_labels(y0_test)

In [ ]:
from keras.losses import CategoricalCrossentropy
model0 = Sequential()
model0.add(LSTM(64, input_shape=(75,50), return_sequences=True))
model0.add(Dropout(0.5))
model0.add(LSTM(64, return_sequences=False))
model0.add(Dropout(0.5))
model0.add(Dense(3))
model0.add(Activation('softmax'))
model0.compile(loss = CategoricalCrossentropy(label_smoothing = 0.1), optimizer='adam', metrics=['accuracy'])
model0.summary()

In [ ]:
hist = model0.fit(embedding_matrix_train0, y_train_hot_smooth, epochs=20, batch_size=32, shuffle=True, validation_split=0.1)

In [ ]:
model0.evaluate(embedding_matrix_test0, y0_test)

In [ ]:
model0.evaluate(embedding_matrix_test0, y_test_hot_smooth)